In [136]:
import pandas as pd
import matplotlib.pyplot as plt
# from utils.Stats import KeyPointsData,AllStats,AggregateStats
import numpy as np
import os
import csv
import warnings
from utils.Builder import Builder
from utils.Tools import RallyExtraction,split_string,ServeTranslate
from utils.SeqMining import GetStrokes, Sequencer

In [137]:
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
points = pd.read_csv('ProjData/Points.csv',encoding='unicode_escape',quoting=csv.QUOTE_NONE)

In [138]:
# Builder("Roger Federer",(2006,2008))

In [139]:
FedererGames = pd.read_csv('Federer_data\FedererStart.csv')
FedererPoints = pd.read_csv('Federer_data\PointsFedererStart.csv')

In [140]:
FedererPoints = pd.merge(FedererPoints,FedererGames[['match_id','Round']],how='left',on='match_id')
FedererPoints = pd.merge(FedererPoints,FedererGames[['match_id','Best of']],how='left',on='match_id')

In [141]:
a = ['S','R','Q','P',' ']
FedererPoints = FedererPoints[~FedererPoints['1st'].isin(a)]
FedererPoints = FedererPoints[~FedererPoints['2nd'].isin(a)]

In [142]:
def SnV_Situation(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        serve = r[:2]
        if(serve[-1] == "+"):
            Situation.append(1)
        else:
            Situation.append(0)
    
    return Situation

def Net_Situation(data):
    SituationS = []
    SituationR = []
    Rallys = RallyExtraction(data)
    
    for r in Rallys:
        split = split_string(r)
        splitS = split[::2]
        splitR = split[1::2]
        foundS = 0
        for s in splitS:
            if(s.find('+') != -1):
                foundS = 1
                break
            
        if(foundS == 1):
            SituationS.append(1)
        else:
            SituationS.append(0)

        foundR = 0
        for s_r in splitR:
            
            if(s_r.find('+') != -1):
                foundR = 1
                break
            
        if(foundR == 1):
            SituationR.append(1)
        else:
            SituationR.append(0)
    
    return SituationS,SituationR

def Serve_Situation(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        if(r[0] == "4"):
            Situation.append("Wide")
        elif(r[0] == "5"):
            Situation.append("Body")
        else:
            Situation.append("Down The T")
    
    return Situation

def RallySize(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        size = len(split_string(r))
        if(size == 1):
            Situation.append(size)
        else:
            Situation.append(size-1)

    
    return Situation

def FinalSHOT_Type_N_Direction(data):
    Type = []
    Direction = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        r = ServeTranslate(r)
        print(r)
        if(len(r)<=3):
            Type.append('No Final Shot')
            Direction.append('No Final Shot')
        else:
            Return = split_string(r)[-1]
            print(Return)
            Type.append(Return[0])
            if(len(Return) > 1):
                Direction.append(Return[1])
            else:
                Direction.append('No Direction')

    return Type,Direction

def ReturnType_N_Direction(data):
    Type = []
    Direction = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        r = ServeTranslate(r)
        print(r)
        if(len(r)<=3):
            Type.append('No Return')
            Direction.append('No Return')
        else:
            Return = split_string(r)[1]
            print(Return)
            Type.append(Return[0])
            if(len(Return) > 1):
                Direction.append(Return[1])
            else:
                Direction.append('No Direction')

    return Type,Direction

def PointSize(data):
    data.loc[(data['Rally Size'] <= 3), ['ShortPoint']] = 1
    data.loc[((data['Rally Size'] > 3) & (data['Rally Size'] <=7)) , ['MediumPoint']] = 1
    data.loc[(data['Rally Size'] > 7), ['LargePoint']] = 1


def GameAhead(data):
    Situation = []
    for _,row in data.iterrows():
        split = row['Pts'].split('-')
        if(row['Server'] == True):
            if(split[0] > split[1]):
                Situation.append(1)
            elif(split[0] == split[1]):
                Situation.append(0)
            else:
                Situation.append(-1)
        else:
            if(split[0] > split[1]):
                Situation.append(-1)
            elif(split[0] == split[1]):
                Situation.append(0)
            else:
                Situation.append(1)
    
    return Situation

def SetAhead(data,player):
    data.loc[((data['Player 1'] == player)&(data['Gm1'] > data['Gm2'])), ['isAheadSet']] = 1
    data.loc[((data['Player 1'] == player)&(data['Gm1'] == data['Gm2'])), ['isAheadSet']] = 0
    data.loc[((data['Player 1'] == player)&(data['Gm1'] < data['Gm2'])), ['isAheadSet']] = -1
    data.loc[((data['Player 2'] == player)&(data['Gm1'] > data['Gm2'])), ['isAheadSet']] = -1
    data.loc[((data['Player 2'] == player)&(data['Gm1'] == data['Gm2'])), ['isAheadSet']] = 0
    data.loc[((data['Player 2'] == player)&(data['Gm1'] < data['Gm2'])), ['isAheadSet']] = 1
    
def MatchAhead(data,player):
    data.loc[((data['Player 1'] == player)&(data['Set1'] > data['Set2'])), ['isAheadMatch']] = 1
    data.loc[((data['Player 1'] == player)&(data['Set1'] == data['Set2'])), ['isAheadMatch']] = 0
    data.loc[((data['Player 1'] == player)&(data['Set1'] < data['Set2'])), ['isAheadMatch']] = -1
    data.loc[((data['Player 2'] == player)&(data['Set1'] > data['Set2'])), ['isAheadMatch']] = -1
    data.loc[((data['Player 2'] == player)&(data['Set1'] == data['Set2'])), ['isAheadMatch']] = 0
    data.loc[((data['Player 2'] == player)&(data['Set1'] < data['Set2'])), ['isAheadMatch']] = 1

def ContainFH(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        if(r.contains('f')):
            pass
    
    return Situation

    

In [143]:
from collections import Counter

In [144]:
FedererPoints['SnV'] = SnV_Situation(FedererPoints)
NetServer, NetReceiver = Net_Situation(FedererPoints)
FedererPoints['Net_Server'] = NetServer
FedererPoints['Net_Returner'] = NetReceiver
FedererPoints['Serve'] = Serve_Situation(FedererPoints)
FedererPoints['Rally Size'] = RallySize(FedererPoints)
FedererPoints['ShortPoint'] = 0
FedererPoints['MediumPoint'] = 0
FedererPoints['LargePoint'] = 0
PointSize(FedererPoints)
FedererPoints['isGameAhead'] = GameAhead(FedererPoints)
FedererPoints['isAheadSet'] = 0
FedererPoints['isAheadMatch'] = 0
SetAhead(FedererPoints,'Roger Federer')
MatchAhead(FedererPoints,'Roger Federer')
FedererPoints['DecisiveSet'] = 0
FedererPoints.loc[((FedererPoints['Best of'] == 5) & (FedererPoints['Set#'] == 5)), ['DecisiveSet']] = 1
FedererPoints.loc[((FedererPoints['Best of'] == 3) & (FedererPoints['Set#'] == 3)), ['DecisiveSet']] = 1
FedererPoints['Tiebreaker'] = 0
FedererPoints.loc[((FedererPoints['Gm1'] == 6) & (FedererPoints['Gm2'] == 6)), ['Tiebreaker']] = 1



In [145]:
Type, Direction = ReturnType_N_Direction(FedererPoints)
FedererPoints['Return Type'] = Type
FedererPoints['Return Direction'] = Direction


6*
4b28f2n@
b28
6*
4b29f3b2f1f3n#
b29
5s28f1f2f2n@
s28
4b29f2f3s2n@
b29
6s2d#
s2
4b2d#
b2
6b28f1f1f2b2f1f1f2b2d@
b28
4s38b3b1*
s38
5b28b2b3b3b2f2n#
b28
4s38f3*
s38
5b28b3b1*
b28
4b2n#
b2
5b38s2f3*
b38
6f38b3b2d@
f38
5s28b2b2b3b3b3b3b2f2f2b1f3n#
s28
4b28b2d@
b28
5f29b3b2f3s3b2f1f3s3w@
f29
4s1n#
s1
4f2n#
f2
4s37v1*
s37
5f2n#
f2
6f28f3n@
f28
5b38s3s3b3s3w@
b38
4b#
5b37b3b1f2f3m1o2*
b37
5s38f3b2f2b3b3d@
s38
4f28b2b2b3b3b1f2b3b1f2d#
f28
4s28v1*
s28
5b29f1f2f2n@
b29
4s37s1f1v2b1*
s37
4f18f3b3w#
f18
4s37b1d#
s37
5d
4s38b3w@
s38
6b19f1f3n#
b19
4b38b2f2f2f1w@
b38
5b28f2b2f3b3b2f1f1f3s3s3s1w#
b28
4b28f2f3b2b3s2f2f3b3b2f2f1f1f1*
b28
5s17f3d@
s17
5f28f3b2n@
f28
5s38b3w@
s38
4b2d#
b2
6b27f3b2n#
b27
4b3w@
b3
4f27b3d@
f27
5f28f3n@
f28
5s3n#
s3
4b28j1f2b3s3f3s3f1l2o2f2b1d@
b28
5b18*
b18*
4*
6*
4f28f3*
f28
4b2d#
b2
4f28b2f3b2f3s3b2f3s2f3m3t3z1*
f28
4s27s3b1*
s27
5b28b3b3b3b1n@
b28
4b37f1f2f2b1*
b37
5s3w@
s3
6*
5b28b3b2n@
b28
4s38f3b3n#
s38
5b28f3b2f1f2b2b3b1d@
b28
6*
4*
5f29f1f1f3s2b3b3b3b3b3b3b3b1*
f2

In [146]:
Type, Direction = FinalSHOT_Type_N_Direction(FedererPoints)
# FedererPoints['Return Type'] = Type
# FedererPoints['Return Direction'] = Direction

6*
4b28f2n@
n@
6*
4b29f3b2f1f3n#
n#
5s28f1f2f2n@
n@
4b29f2f3s2n@
n@
6s2d#
d#
4b2d#
d#
6b28f1f1f2b2f1f1f2b2d@
d@
4s38b3b1*
b1*
5b28b2b3b3b2f2n#
n#
4s38f3*
f3*
5b28b3b1*
b1*
4b2n#
n#
5b38s2f3*
f3*
6f38b3b2d@
d@
5s28b2b2b3b3b3b3b2f2f2b1f3n#
n#
4b28b2d@
d@
5f29b3b2f3s3b2f1f3s3w@
w@
4s1n#
n#
4f2n#
n#
4s37v1*
v1*
5f2n#
n#
6f28f3n@
n@
5b38s3s3b3s3w@
w@
4b#
5b37b3b1f2f3m1o2*
o2*
5s38f3b2f2b3b3d@
d@
4f28b2b2b3b3b1f2b3b1f2d#
d#
4s28v1*
v1*
5b29f1f2f2n@
n@
4s37s1f1v2b1*
b1*
4f18f3b3w#
w#
4s37b1d#
d#
5d
4s38b3w@
w@
6b19f1f3n#
n#
4b38b2f2f2f1w@
w@
5b28f2b2f3b3b2f1f1f3s3s3s1w#
w#
4b28f2f3b2b3s2f2f3b3b2f2f1f1f1*
f1*
5s17f3d@
d@
5f28f3b2n@
n@
5s38b3w@
w@
4b2d#
d#
6b27f3b2n#
n#
4b3w@
w@
4f27b3d@
d@
5f28f3n@
n@
5s3n#
n#
4b28j1f2b3s3f3s3f1l2o2f2b1d@
d@
5b18*
b18*
4*
6*
4f28f3*
f3*
4b2d#
d#
4f28b2f3b2f3s3b2f3s2f3m3t3z1*
z1*
4s27s3b1*
b1*
5b28b3b3b3b1n@
n@
4b37f1f2f2b1*
b1*
5s3w@
w@
6*
5b28b3b2n@
n@
4s38f3b3n#
n#
5b28f3b2f1f2b2b3b1d@
d@
6*
4*
5f29f1f1f3s2b3b3b3b3b3b3b3b1*
b1*
4f27f1f1f2f3b1v1*
v1*
4b38b1*


In [147]:
dic = GetStrokes()

In [148]:
FedererPoints['Return Type'] = FedererPoints['Return Type'].map(dic)
FedererPoints['Return Direction'] = FedererPoints['Return Direction'].map(dic)

In [149]:
FedererPoints['Pts'].str.split('-')

0         [0, 0]
1        [15, 0]
2       [15, 15]
3       [30, 15]
4       [40, 15]
          ...   
7802      [4, 3]
7803      [5, 3]
7804      [4, 5]
7805      [5, 5]
7806      [5, 6]
Name: Pts, Length: 7722, dtype: object

In [150]:
FedererPoints.loc[FedererPoints['Pt']]

,match_id,Pt,Set1,Set2,Set#,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,Player 1,Player 2,Surface,Victor,Server,GameWinner,SetWinner,Ace,Winner,Unforced Error,Forced Error,Double Fault,Round,Best of,SnV,Net_Server,Net_Returner,Serve,Rally Size,ShortPoint,MediumPoint,LargePoint,isGameAhead,isAheadSet,isAheadMatch,DecisiveSet,Tiebreaker,Return Type,Return Direction
1,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,2,0,0,1,0.0,0.0,15-0,1.0,True,1,6n,4b28f2n@,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Wide,3,1,0,0,1,0,0,0,0,Backhand,Meio
2,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,3,0,0,1,0.0,0.0,15-15,1.0,True,1,6*,False,Blake challenged incorrectly,1,Roger Federer,James Blake,Hard,True,True,True,True,True,False,False,False,False,F,5,0,0,0,Down The T,1,1,0,0,0,0,0,0,0,No Return,No Return
3,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,4,0,0,1,0.0,0.0,30-15,1.0,True,1,6d,4b29f3b2f1f3n#,False,1,Roger Federer,James Blake,Hard,True,True,True,True,False,False,False,True,False,F,5,0,0,0,Wide,6,0,1,0,1,0,0,0,0,Backhand,Meio
4,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,5,0,0,1,0.0,0.0,40-15,1.0,True,1,5s28f1f2f2n@,False,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Body,5,0,1,0,1,0,0,0,0,BH Slice,Meio
5,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,6,0,0,1,0.0,0.0,40-30,1.0,True,1,4w,4b29f2f3s2n@,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Wide,5,0,1,0,1,0,0,0,0,Backhand,Meio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,162,2,0,3,5.0,3.0,30-0,24.0,True,2,4n,5b38b3*,False,2,Roger Federer,James Blake,Hard,False,False,False,True,False,True,False,False,False,F,5,0,0,0,Body,2,1,0,0,-1,1,1,0,0,Backhand,Paralelo
162,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,163,2,0,3,5.0,3.0,40-0,24.0,True,2,5d,4s3n#,False,2,Roger Federer,James Blake,Hard,False,False,False,True,False,False,False,True,False,F,5,0,0,0,Wide,2,1,0,0,-1,1,1,0,0,BH Slice,Paralelo
163,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,164,2,0,3,5.0,4.0,0-0,25.0,True,1,6n,5b1w#,False,1,Roger Federer,James Blake,Hard,True,True,True,True,False,False,False,True,False,F,5,0,0,0,Body,2,1,0,0,0,1,1,0,0,Backhand,Cruzado
164,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,165,2,0,3,5.0,4.0,15-0,25.0,True,1,4b#,False,False,1,Roger Federer,James Blake,Hard,True,True,True,True,False,False,False,True,False,F,5,0,0,0,Wide,1,1,0,0,1,1,1,0,0,No Return,No Return


In [151]:
FedererPoints[['Last_5_pts','Last_4_pts','Last_3_pts','Last_2_pts','LastPt']] = 0
FedererPoints[['Forehand,','Backhand','FH Slice','BH Slice','FH Volley','BH Volley','forehand drop shot','backhand drop shot','forehand lob','backhand lob','forehand half-volley','backhand half-volley','forehand swinging volley','backhand swinging volley','unknown','trick shot','any']] = 0

In [152]:
aux = ['Forehand,','Backhand','FH Slice','BH Slice','FH Volley','BH Volley','forehand drop shot','backhand drop shot','forehand lob','backhand lob','forehand half-volley','backhand half-volley','forehand swinging volley','backhand swinging volley','unknown','trick shot','any']

In [153]:
R = RallyExtraction(FedererPoints)
R

['6*',
 '4b28f2n@',
 '6*',
 '4b29f3b2f1f3n#',
 '5s28f1f2f2n@',
 '4b29f2f3s2n@',
 '6s2d#',
 '4b2d#',
 '6b28f1f1f2b2f1f1f2b2d@',
 '4s38b3b1*',
 '5b28b2b3b3b2f2n#',
 '4s38f3*',
 '5b28b3b1*',
 '4b2n#',
 '5b38s2f+3*',
 '6f38b3b2d@',
 '5s28b2b2b3b3b3b3b2f2f2b1f3n#',
 '4b28b2d@',
 '5f29b3b2f3s3b2f1f3s3w@',
 'c4s1n#',
 '4f2n#',
 '4+s37v1*',
 '5f2n#',
 '6f28f3n@',
 '5b38s3s3b3s3w@',
 '4b#',
 '5b37b3b1f2f+3m1o2*',
 '5s38f3b2f2b3b3d@',
 '4f28b2b2b3b3b1f2b3b1f2d#',
 '4+s28v1*',
 '5b29f1f2f2n@',
 '4s37s+1f1v2b1*',
 '4f18f3b3w#',
 '4s37b1d#',
 '5d',
 '4s38b3w@',
 '6b19f1f3n#',
 '4b38b2f2f2f1w@',
 '5b28f2b2f3b3b2f1f1f3s3s+3s1w#',
 '4b28f2f3b2b3s2f2f3b3b2f2f1f1f1*',
 '5s17f3d@',
 '5f28f+3b2n@',
 '5s38b3w@',
 '4b2d#',
 '6b27f+3b2n#',
 '4b3w@',
 '4f27b3d@',
 '5f28f3n@',
 '5s3n#',
 '4b28j1f2b3s3f3s3f1l2o=2f2b1d@',
 '5b18*',
 '4*',
 '6*',
 '4f28f3*',
 '4b2d#',
 'c4f28b2f3b2f3s3b2f3s2f+3m+3t3z1*',
 '4s27s+3b1*',
 '5b28b3b3b3b1n@',
 '4b37f1f2f2b1*',
 '5s3w@',
 'c6*',
 '5b28b3b2n@',
 '4s38f+3b3n#',
 '5b28f3b

In [156]:
FedererPoints

,match_id,Pt,Set1,Set2,Set#,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,Player 1,Player 2,Surface,Victor,Server,GameWinner,SetWinner,Ace,Winner,Unforced Error,Forced Error,Double Fault,Round,Best of,SnV,Net_Server,Net_Returner,Serve,Rally Size,ShortPoint,MediumPoint,LargePoint,isGameAhead,isAheadSet,isAheadMatch,DecisiveSet,Tiebreaker,Return Type,Return Direction,Last_5_pts,Last_4_pts,Last_3_pts,Last_2_pts,LastPt,"Forehand,",Backhand,FH Slice,BH Slice,FH Volley,BH Volley,forehand drop shot,backhand drop shot,forehand lob,backhand lob,forehand half-volley,backhand half-volley,forehand swinging volley,backhand swinging volley,unknown,trick shot,any
0,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,1,0,0,1,0.0,0.0,0-0,1.0,True,1,6*,False,False,1,Roger Federer,James Blake,Hard,True,True,True,True,True,False,False,False,False,F,5,0,0,0,Down The T,1,1,0,0,0,0,0,0,0,No Return,No Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,2,0,0,1,0.0,0.0,15-0,1.0,True,1,6n,4b28f2n@,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Wide,3,1,0,0,1,0,0,0,0,Backhand,Meio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,3,0,0,1,0.0,0.0,15-15,1.0,True,1,6*,False,Blake challenged incorrectly,1,Roger Federer,James Blake,Hard,True,True,True,True,True,False,False,False,False,F,5,0,0,0,Down The T,1,1,0,0,0,0,0,0,0,No Return,No Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,4,0,0,1,0.0,0.0,30-15,1.0,True,1,6d,4b29f3b2f1f3n#,False,1,Roger Federer,James Blake,Hard,True,True,True,True,False,False,False,True,False,F,5,0,0,0,Wide,6,0,1,0,1,0,0,0,0,Backhand,Meio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,5,0,0,1,0.0,0.0,40-15,1.0,True,1,5s28f1f2f2n@,False,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Body,5,0,1,0,1,0,0,0,0,BH Slice,Meio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,161,1,0,2,6.0,6.0,4-3,22.0,True,2,6n,6f38b2f+1f3*,False,2,Roger Federer,Gael Monfils,Hard,False,False,True,True,False,True,False,False,False,F,3,0,0,1,Down The T,4,0,1,0,-1,0,1,0,1,Forehand,Paralelo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7803,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,162,1,0,2,6.0,6.0,5-3,22.0,True,2,6d,5f28f2f3*,False,1,Roger Federer,Gael Monfils,Hard,True,False,True,True,False,True,False,False,False,F,3,0,0,0,Body,3,1,0,0,-1,0,1,0,1,Forehand,Meio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7804,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,163,1,0,2,6.0,6.0,4-5,22.0,True,1,4b2d#,False,False,1,Roger Federer,Gael Monfils,Hard,True,True,True,True,False,False,False,True,False,F,3,0,0,0,Wide,2,1,0,0,-1,0,1,0,1,Backhand,Meio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7805,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,164,1,0,2,6.0,6.0,5-5,22.0,True,1,6b27f+3b2v1*,False,False,1,Roger Federer,Gael Monfils,Hard,True,True,True,True,False,True,False,False,False,F,3,0,1,0,Down The T,4,0,1,0,0,0,1,0,1,Backhand,Meio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
